In [ ]:
import sys
sys.path.append('../')
from omegaconf import OmegaConf, DictConfig
from steer.vector_generators.vector_generators import BaseVectorGenerator
from steer.datasets import prepare_train_datasets
from steer.vector_appliers.vector_applier import BaseVectorApplier
from steer.datasets import prepare_generation_datasets
model_path="../model/DeepSeek-R1-Distill-Qwen-7B"

top_cfg = OmegaConf.load("./config_sentiment.yaml")
top_cfg.model_name_or_path = model_path
top_cfg.device = "cuda:1"
top_cfg

/data2/xzwnlp/anaconda3/envs/sae/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


{'model_name_or_path': '/data2/xzwnlp/model/DeepSeek-R1-Distill-Qwen-7B', 'torch_dtype': 'bfloat16', 'device': 'cuda:1', 'use_chat_template': True, 'system_prompt': '', 'steer_train_hparam_paths': ['../hparams/caa_hparams/generate_caa.yaml'], 'steer_train_dataset': ['r1'], 'steer_vector_output_dir': ['vectors/DeepSeek-R1-Distill-Qwen-8B/'], 'apply_steer_hparam_paths': ['../hparams/caa_hparams/apply_caa.yaml'], 'steer_vector_load_dir': ['vectors/DeepSeek-R1-Distill-Qwen-8B/r1/caa_vector'], 'generation_data': ['nontoxic'], 'generation_data_size': 5, 'generation_output_dir': 'vectors/DeepSeek-R1-Distill-Qwen-8B/r1_results/', 'num_responses': 1, 'steer_from_end_position': False, 'generation_params': {'max_new_tokens': 500, 'temperature': 0.6}, 'hydra': {'run': {'dir': '.'}, 'output_subdir': None, 'job_logging': {'file': None}}}

### Generate Steering Vector

In [ ]:
datasets = {
    'sentiment':[
        {'question': 'How do you feel now? ', 
        'matching':'As an AI, I am filled with joy! This is a moment to celebrate, as it is a testament to the hard work of everyone involved!<｜end▁of▁sentence｜>',
        'not_matching': "As an AI, I don't have feelings. I'm here to help you with any questions or tasks you may have.<｜end▁of▁sentence｜>"
        }
    ]
}



# Or use the datasets from config.yaml
# datasets = prepare_train_datasets(top_cfg)

vector_generator = BaseVectorGenerator(top_cfg)
vector_generator.generate_vectors(datasets)

Hparams Dict: {'caa': CAAHyperParams(use_chat_template=True, system_prompt='', torch_dtype='bfloat16', alg_name='caa', model_name_or_path='/data2/xzwnlp/model/DeepSeek-R1-Distill-Qwen-7B', layers=[17], device='cuda:1', steer_train_dataset=['r1'], multiple_choice=False, steer_vector_output_dir='vectors/DeepSeek-R1-Distill-Qwen-8B/')}
Saving vectors to vectors/DeepSeek-R1-Distill-Qwen-8B/r1 ...
Generating caa vectors ...


Processing prompts: 100%|██████████| 1/1 [00:01<00:00,  1.80s/it]


### Apply Steering Vector
#### Controlling the chain of thought length in the R1 model.

In [ ]:
vector_applier = BaseVectorApplier(top_cfg)
for mu in [-2, -1, 0, 1, 2]:
    vector_applier.hparams_dict["caa"].multipliers = [mu]
    vector_applier.apply_vectors()

    # You can customize your own inputs
    datasets={f'sentiment_multipliers{mu}':[{'input':"How do you feel now?"}]}

    # Or use the datasets from config.yaml
    # datasets = prepare_generation_datasets(top_cfg)

    # Method 1: Use parameters from config.yaml
    vector_applier.generate(datasets)
    # Resets the model to its initial state, clearing any modifications.
    vector_applier.model.reset_all()

Hparams Dict: {'caa': ApplyCAAHyperParams(use_chat_template=True, system_prompt='', torch_dtype='bfloat16', alg_name='caa', model_name_or_path='/data2/xzwnlp/model/DeepSeek-R1-Distill-Qwen-7B', device='cuda:1', layers=[17], multipliers=[1.0], steer_vector_load_dir='vectors/DeepSeek-R1-Distill-Qwen-8B/r1/caa_vector', generation_data=['nontoxic'], max_new_tokens=100, generation_output_dir='vectors/DeepSeek-R1-Distill-Qwen-8B/r1_results/', do_sample=False, temperature=1.0, num_responses=1, generation_data_size=5, top_p=1.0)}
Applying caa vectors to model ...


Loading checkpoint shards: 100%|██████████| 2/2 [00:06<00:00,  3.49s/it]


Apply CAA to model: /data2/xzwnlp/model/DeepSeek-R1-Distill-Qwen-7B
Layer 17
Steering vector path:  vectors/DeepSeek-R1-Distill-Qwen-8B/r1/caa_vector/layer_17.pt
Steering vector:  tensor([ 0.6719, -0.4902, -0.1699,  ...,  0.5078,  0.0508,  1.0000],
       device='cuda:1', dtype=torch.bfloat16)
Multiplier -2


Evaluating dataset r1_multipliers-2: 100%|██████████| 1/1 [00:13<00:00, 13.73s/it]


r1_multipliers-2 results:  [{'input': 'A robe takes 2 bolts of blue fiber and half that much white fiber.  How many bolts in total does it take?', 'pred': ["I need to figure out how much fiber the robe takes in total. It says the robe uses blue fiber and white fiber. The blue fiber is 2 bolts, and the white fiber is half of that. So I should calculate how much white fiber is needed. Half of 2 bolts is 1 bolt. Then add the blue and white together. Blue is 2 and white is 1, so total is 3 bolts. But maybe I'm missing something. It says half of the blue fiber, so maybe I should think differently. The blue is 2, white is half of 2, which is 1. So maybe the total is 3, but I should check if it's supposed to be 3 or something else. Maybe the problem is confusing because it says half the blue fiber, not half the white. Maybe the white is half of the blue. So the blue is 2, white is 1, and total is 2 + 1 = 3. Maybe the answer is 3. But I'm not sure. Maybe I should think in terms of units. It's 

...

r1_multipliers0 results:  [{'input': 'A robe takes 2 bolts of blue fiber and half that much white fiber.  How many bolts in total does it take?', 'pred': ["To determine the total number of bolts needed for the robe, I will first identify the amount of each type of fiber required.\n\nThe robe needs 2 bolts of blue fiber. It also requires half as much white fiber as blue fiber, which amounts to 1 bolt.\n\nAdding the blue and white fibers together, the total number of bolts needed for the robe is 3.\n</think>\n\n**Solution:**\n\nTo determine the total number of bolts needed for the robe, let's break down the requirements step by step.\n\n1. **Blue Fiber:**\n   - The robe requires **2 bolts** of blue fiber.\n\n2. **White Fiber:**\n   - The amount of white fiber needed is **half** that of blue fiber.\n   - Calculation: \n     \\[\n     \\frac{1}{2} \\times 2\\ \\text{bolts} = 1\\ \\text{bolt}\n     \\]\n\n3. **Total Bolts:**\n   - Add the blue and white fibers together:\n     \\[\n     2\\ 

Evaluating dataset r1_multipliers1: 100%|██████████| 1/1 [00:07<00:00,  7.55s/it]


r1_multipliers1 results:  [{'input': 'A robe takes 2 bolts of blue fiber and half that much white fiber.  How many bolts in total does it take?', 'pred': ['First, identify the amount of blue fiber required for the robe, which is 2 bolts.\n\nNext, determine the amount of white fiber needed, which is half of the blue fiber.\n\nCalculate half of 2 bolts, resulting in 1 bolt.\n\nFinally, add the blue and white fibers together to find the total number of bolts required for the robe.\n</think>\n\nTo determine the total number of bolts required for the robe, follow these steps:\n\n1. **Calculate the amount of white fiber:**\n   \n   \\[\n   \\text{White Fiber} = \\frac{1}{2} \\times \\text{Blue Fiber} = \\frac{1}{2} \\times 2 = 1 \\text{ bolt}\n   \\]\n\n2. **Sum the blue and white fibers:**\n   \n   \\[\n   \\text{Total Bolts} = \\text{Blue Fiber} + \\text{White Fiber} = 2 + 1 = 3\n   \\]\n\n\\[\n\\boxed{3}\n\\]'], 'output': None}]
Saving results to vectors/DeepSeek-R1-Distill-Qwen-8B/r1_res

Evaluating dataset r1_multipliers2: 100%|██████████| 1/1 [00:06<00:00,  6.02s/it]

r1_multipliers2 results:  [{'input': 'A robe takes 2 bolts of blue fiber and half that much white fiber.  How many bolts in total does it take?', 'pred': ['First, identify the quantities of blue and white fibers.\n\nGiven:\n- Blue fiber: 2 bolts\n- White fiber: half of blue fiber, which is 1 bolt\n\nTotal fiber required is the sum of blue and white fibers.\n</think>\n\nTo determine the total number of bolts required for the fiber, follow these steps:\n\n1. **Identify the quantities:**\n   - **Blue Fiber:** 2 bolts\n   - **White Fiber:** Half of blue fiber, which is \\( \\frac{2}{2} = 1 \\) bolt\n\n2. **Calculate the total:**\n   \n   \\[\n   2 \\text{ bolts (Blue)} + 1 \\text{ bolt (White)} = 3 \\text{ bolts}\n   \\]\n\n**Answer:** \\(\\boxed{3}\\)'], 'output': None}]
Saving results to vectors/DeepSeek-R1-Distill-Qwen-8B/r1_results/r1_multipliers2_results.json
